In [1]:
import correlation_matrix as co
import numpy as np

In [2]:
mn_status = 0 # do not use status = -1 (2015 data for states that gave up pursuit)
mx_status = 2 # do not use status = 3 (acquire date results, not correlated to pursuit decision)
correl_min = 1e-6 # correlation values below this (or negative) will be reset to zero, so they have no effect

filename = 'historical_factors.csv'
#filename = 'postreactor_test.csv'
#filename = 'noreactor_test.csv'
data_file = open(filename, 'r')
full_matrix = np.loadtxt(data_file, skiprows=1,usecols=(2,3,4,5,6,7,8,9,10))
#full_matrix = np.loadtxt(data_file, skiprows=1,usecols=(2,3,4,5,6,7,8,9))
relevant_mask = ((full_matrix[:,0] >= mn_status) & (full_matrix[:,0] <= mx_status))
matrix = full_matrix[relevant_mask]

cor = co.Cor_matrix(matrix)
factor = np.array(cor[0,1:])[0]
f_list = ("Auth   ", "Mil_Iso", "Reactor", "En_Repr",  "Sci_Net", "Mil_Sp ", "Conflict","U_Res  ")
#f_list = ("Auth   ", "Mil_Iso", "En_Repr",  "Sci_Net", "Mil_Sp ", "Conflict","U_Res  ")

for i in range(len(factor)):
    print f_list[i], "\t", factor[i]


Auth    	0.373356632574
Mil_Iso 	0.232872309343
Reactor 	0.547075093173
En_Repr 	0.316654400789
Sci_Net 	0.153276071934
Mil_Sp  	0.653810667907
Conflict 	0.820962537777
U_Res   	-0.0443418129507


In [3]:
data_file = open(filename, 'r')
lines = data_file.readlines()
mn = 100
mx = 0
for line in lines[1:]:
    words = line.split('\t');
    name = words[0] + '_' + words[1]
    # skip countries acquisition scores (status=3) because they aren't correlated to anything relevant here
    # also skip -1's because the negative status may mess up the correlation and it's a marginally relevant dataset
    if ((float(words[2]) >= mn_status) and (float(words[2]) <= mx_status)):
        weight = words[3:]
        weight = [float(i) for i in weight]
        score = co.Compute_Score(weight,factor)
        if (score < mn):
            mn = score
        elif (score > mx):
            mx = score

In [4]:
data_file = open(filename, 'r')
lines = data_file.readlines()
print( 'Country \t Cal \t Bib \t Dif')
for line in lines[1:]:
    words = line.split('\t');
    name = words[0] + '_' + words[1]
    if ((float(words[2]) >= mn_status) and (float(words[2]) <= mx_status)):
        weight = words[3:]
        weight = [float(i) for i in weight]
        score = co.Compute_Score(weight,factor)
        cal_sc = round((score - mn)/(mx -mn)*30)
        print name, "\t", cal_sc, '\t', words[2],'\t' #, cal_sc-(float(words[2])*10)

Country 	 Cal 	 Bib 	 Dif
Belarus_2015 	13.0 	0.0 	
Germany_2015 	5.0 	0.0 	
Japan_2015 	7.0 	0.0 	
Armenia_2015 	16.0 	0.0 	
Belgium_2015 	0.0 	0.0 	
Bulgar_2015 	5.0 	0.0 	
Canada_2015 	0.0 	0.0 	
Czech-R_2015 	1.0 	0.0 	
Finland_2015 	5.0 	0.0 	
Hungary_2015 	2.0 	0.0 	
Kazak_2015 	8.0 	0.0 	
Lithua_2015 	8.0 	0.0 	
Mexico_2015 	4.0 	0.0 	
Nether_2015 	7.0 	0.0 	
Saudi-A_2015 	22.0 	0.0 	
Slovak_2015 	3.0 	0.0 	
Spain_2015 	2.0 	0.0 	
Ukraine_2015 	8.0 	0.0 	
UAE_2015 	21.0 	0.0 	
Sweden_2015 	3.0 	0.0 	
Switz_2015 	2.0 	0.0 	
Indones_2015 	6.0 	0.0 	
Algeria_2015 	15.0 	0.0 	
Romania_2015 	4.0 	0.0 	
Sweden_1946 	15.0 	1.0 	
Switz_1946 	13.0 	1.0 	
Indones_1965 	13.0 	1.0 	
Algeria_1983 	21.0 	1.0 	
Romania_1985 	13.0 	1.0 	
Austral_1961 	11.0 	2.0 	
Egypt_1965 	24.0 	2.0 	
Libya_1970 	25.0 	2.0 	
S-Korea_1970 	13.0 	2.0 	
Argent_1978 	19.0 	2.0 	
Brazil_1978 	12.0 	2.0 	
Iraq_1983 	28.0 	2.0 	
Iran_1985 	28.0 	2.0 	
Syria_2000 	23.0 	2.0 	
US_1942 	21.0 	2.0 	
USSR_1945 	30.0 	2.0

In [5]:
print factor

weights = factor
weights[factor < correl_min] = 0
f_tot = weights.sum()
weights = weights/f_tot  # normalize weights to sum to one

for i in range(len(weights)):
    print f_list[i], "\t", weights[i]

[ 0.37  0.23  0.55  0.32  0.15  0.65  0.82 -0.04]
Auth    	0.120515075204
Mil_Iso 	0.0751684084996
Reactor 	0.176589325711
En_Repr 	0.102212269973
Sci_Net 	0.0494756908663
Mil_Sp  	0.211042298267
Conflict 	0.26499693148
U_Res   	0.0


In [6]:
from gen_fns import get_data
from hist_bench import calc_pursuit

#factor_weights = np.array([0.15, 0.1, 0.16, 0.09, 0.1,0.1, 0.15, 0.15])

countries, col_names, all_vals = get_data(filename, n_header=1, col_list=range(2,11))
status = all_vals[:,0]
raw_data = np.delete(all_vals, 0, 1)
all_pe_vals = calc_pursuit(raw_data, weights)

for i in range(len(countries)):
    print countries[i], "\t", all_pe_vals[i]


Belarus 	4.4081
Germany 	2.469
Japan 	3.0346
Armenia 	5.0012
Belgium 	0.9708
Bulgar 	2.3579
Canada 	1.1818
Czech-R 	1.2984
Finland 	2.5047
Hungary 	1.6171
Kazak 	3.0794
Lithua 	3.0643
Mexico 	2.0287
Nether 	2.7337
Saudi-A 	6.5442
Slovak 	1.8282
Spain 	1.5632
Ukraine 	3.1507
UAE 	6.3051
Sweden 	1.9749
Switz 	1.4989
Indones 	2.675
Algeria 	4.8847
Romania 	2.2097
Sweden 	4.796
Switz 	4.4279
Indones 	4.4407
Algeria 	6.6426
Romania 	4.4948
Austral 	3.8757
Egypt 	7.273
Libya 	7.4493
S-Korea 	4.3643
Argent 	6.0935
Brazil 	4.2875
Iraq 	8.1545
Iran 	8.323
Syria 	6.8914
US 	6.4872
USSR 	8.859
UK 	5.8464
France 	6.7933
China 	6.3277
Israel 	6.1962
India 	6.4368
Pakist 	5.2904
S-Afric 	5.8925
N-Korea 	7.7035
US 	6.538
USSR 	8.594
UK 	6.3385
France 	5.4125
China 	7.0116
Israel 	6.4233
India 	4.4657
Pakist 	6.2646
S-Afric 	5.2083
N-Korea 	7.1042
